In [ ]:
!git clone https://github.com/takhahashi/conf.git
%cd 'conf'

In [ ]:
!pip install hydra-core --upgrade
!pip install datasets
!pip install transformers
!pip install orjson
!pip install wget
!pip install pytreebank
!pip install accelerate
!pip install -U transformers
!pip install optuna
!pip install fugashi
!pip install unidic_lite
!pip install ipadic
!pip install gpytorch

# 理研のデータセット

## モデル推論

In [ ]:
import os

for model_type in ['hybrid','classification','gaussianregression']:
  for question_id_pref in ['Y14']:
    for question_id_suff in ['1-2_1_3','1-2_2_4','2-1_1_5','2-1_2_3','2-2_1_4','2-2_2_3']:
      for fold in range(5):
        command = f"""
        python eval.py \
        result_savepath="/content/riken_results/{model_type}/{question_id_pref}_{question_id_suff}/fold{fold}" \
        use_trustscore=False \
        data=riken \
        data.question_id_pref="{question_id_pref}" \
        data.question_id_suff="{question_id_suff}" \
        data.score_id='score' \
        data.fold={fold} \
        model.model_type="{model_type}" \
        model.id=0 \
        model.model_name_or_path="/content/{model_type}_bert/{question_id_pref}_{question_id_suff}_score/fold{fold}/id0"
        """

        # Execute the command using os.system()
        print(f"Running fold {fold}...")
        os.system(command)

  for question_id_pref in ['Y15']:
    for question_id_suff in ['2-3_1_5','2-3_2_2','2-3_2_4']:
      for fold in range(5):
        command = f"""
        python eval.py \
        result_savepath="/content/riken_results/{model_type}/{question_id_pref}_{question_id_suff}/fold{fold}" \
        use_trustscore=False \
        data=riken \
        data.question_id_pref="{question_id_pref}" \
        data.question_id_suff="{question_id_suff}" \
        data.score_id='score' \
        data.fold={fold} \
        model.model_type="{model_type}" \
        model.id=0 \
        model.model_name_or_path="/content/{model_type}_bert/{question_id_pref}_{question_id_suff}_score/fold{fold}/id0"
        """

        # Execute the command using os.system()
        print(f"Running fold {fold}...")
        os.system(command)

## 出力結果集計

In [ ]:
import json
import numpy as np
import pandas as pd

hybrid_res = {'qwk':[], 'corr':[], 'roc':[], 'rpp':[]}
classification_res = {'qwk':[], 'corr':[], 'roc':[], 'rpp':[]}
gaussianregression_res = {'qwk':[], 'corr':[], 'roc':[], 'rpp':[]}

for model_type in ['hybrid','classification','gaussianregression']:
  for question_id_pref in ['Y15','Y14']:
    if question_id_pref == 'Y14':
      suff_list = ['1-2_1_3','1-2_2_4','2-1_1_5','2-1_2_3','2-2_1_4','2-2_2_3']
    elif question_id_pref == 'Y15':
      suff_list = ['2-3_1_5','2-3_2_2','2-3_2_4']
    for question_id_suff in suff_list:
      qwk_list = []
      corr_list = []
      roc_list = []
      rpp_list = []
      for fold in range(5):
        with open(f"/content/riken_results/{model_type}/{question_id_pref}_{question_id_suff}/fold{fold}/test_inference.json") as f:
          df = json.load(f)
        qwk_list.append(df['qwk'])
        corr_list.append(df['corr'])
        roc_list.append(df['roc'])
        rpp_list.append(df['rpp'])
      if model_type == 'hybrid':
        hybrid_res['qwk'].append(np.mean(qwk_list))
        hybrid_res['corr'].append(np.mean(corr_list))
        hybrid_res['roc'].append(np.mean(roc_list))
        hybrid_res['rpp'].append(np.mean(rpp_list))
      elif model_type == 'classification':
        classification_res['qwk'].append(np.mean(qwk_list))
        classification_res['corr'].append(np.mean(corr_list))
        classification_res['roc'].append(np.mean(roc_list))
        classification_res['rpp'].append(np.mean(rpp_list))
      elif model_type == 'gaussianregression':
        gaussianregression_res['qwk'].append(np.mean(qwk_list))
        gaussianregression_res['corr'].append(np.mean(corr_list))
        gaussianregression_res['roc'].append(np.mean(roc_list))
        gaussianregression_res['rpp'].append(np.mean(rpp_list))

In [ ]:
qwk_dic = {'hybrid':hybrid_res['qwk'], 'classification':classification_res['qwk'], 'gaussianregression':gaussianregression_res['qwk']}
corr_dic = {'hybrid':hybrid_res['corr'], 'classification':classification_res['corr'], 'gaussianregression':gaussianregression_res['corr']}
roc_dic = {'hybrid':hybrid_res['roc'], 'classification':classification_res['roc'], 'gaussianregression':gaussianregression_res['roc']}
rpp_dic = {'hybrid':hybrid_res['rpp'], 'classification':classification_res['rpp'], 'gaussianregression':gaussianregression_res['rpp']}
qwk_dataframe = pd.DataFrame.from_dict(qwk_dic, orient='index')
corr_dataframe = pd.DataFrame.from_dict(corr_dic, orient='index')
roc_dataframe = pd.DataFrame.from_dict(roc_dic, orient='index')
rpp_dataframe = pd.DataFrame.from_dict(rpp_dic, orient='index')

In [ ]:
qwk_dataframe

,0,1,2,3,4,5,6,7,8
hybrid,0.953704,0.892859,0.718276,0.957030,0.919857,0.863418,0.766404,0.868981,0.912093
classification,0.911404,0.865389,0.617022,0.941123,0.908226,0.824838,0.686225,0.838504,0.891543
gaussianregression,0.943714,0.885649,0.690798,0.958061,0.922988,0.857263,0.729649,0.853839,0.902268


In [ ]:
roc_dataframe

,0,1,2,3,4,5,6,7,8
hybrid,0.782511,0.730179,0.516149,0.728422,0.750852,0.704767,0.604470,0.839548,0.777914
classification,0.703509,0.712453,0.534495,0.722429,0.747512,0.734364,0.620116,0.754155,0.727751
gaussianregression,0.547990,0.595339,0.503915,0.668745,0.647854,0.648011,0.595163,0.723740,0.596585
